In [25]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("Exercicio Intro") \
        .getOrCreate()

df_nomes = spark.read.csv("nomes_aleatorios.txt")
df_nomes.show(5)
df_nomes.printSchema()

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows

root
 |-- _c0: string (nullable = true)



In [26]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nome")
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nome: string (nullable = true)

+-----------------+
|             Nome|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [27]:
from pyspark.sql.functions import rand, when

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when(rand() < 0.33, "Fundamental")
    .when(rand() < 0.66, "Medio")
    .otherwise("Superior")
)
df_nomes.show(5)
df_nomes.printSchema()

+----------------+------------+
|            Nome|Escolaridade|
+----------------+------------+
|  Frances Bennet|    Superior|
|   Jamie Russell| Fundamental|
|  Edward Kistler|       Medio|
|   Sheila Maurer|       Medio|
|Donald Golightly|       Medio|
+----------------+------------+
only showing top 5 rows

root
 |-- Nome: string (nullable = true)
 |-- Escolaridade: string (nullable = false)



In [28]:
paises = ["Argentina", "Bolívia", "Brasil", "Chile", 
          "Colômbia", "Equador", "Guiana", "Guiana Francesa", 
          "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela"]
pais = (
    when(rand() < 0.077, paises[0])
    .when(rand() < 0.154, paises[1])
    .when(rand() < 0.231, paises[2])
    .when(rand() < 0.308, paises[3])
    .when(rand() < 0.385, paises[4])
    .when(rand() < 0.462, paises[5])
    .when(rand() < 0.539, paises[6])
    .when(rand() < 0.616, paises[7])
    .when(rand() < 0.693, paises[8])
    .when(rand() < 0.770, paises[9])
    .when(rand() < 0.847, paises[10])
    .when(rand() < 0.924, paises[11])
    .otherwise(paises[12])
)
df_nomes = df_nomes.withColumn("Pais", pais)
df_nomes.show(5)
df_nomes.printSchema()

+----------------+------------+--------+
|            Nome|Escolaridade|    Pais|
+----------------+------------+--------+
|  Frances Bennet|    Superior|  Guiana|
|   Jamie Russell| Fundamental|Colômbia|
|  Edward Kistler|       Medio|  Brasil|
|   Sheila Maurer|       Medio|  Brasil|
|Donald Golightly|       Medio|Colômbia|
+----------------+------------+--------+
only showing top 5 rows

root
 |-- Nome: string (nullable = true)
 |-- Escolaridade: string (nullable = false)
 |-- Pais: string (nullable = false)



In [29]:
from pyspark.sql.functions import expr

expr_ano = expr(f"CAST({1945} + FLOOR(rand() * (2010 - 1945 + 1)) AS INT)")
df_nomes = df_nomes.withColumn("AnoNascimento", expr_ano)

df_nomes.show(5)
df_nomes.printSchema()

+----------------+------------+--------+-------------+
|            Nome|Escolaridade|    Pais|AnoNascimento|
+----------------+------------+--------+-------------+
|  Frances Bennet|    Superior|  Guiana|         1995|
|   Jamie Russell| Fundamental|Colômbia|         1969|
|  Edward Kistler|       Medio|  Brasil|         1996|
|   Sheila Maurer|       Medio|  Brasil|         2010|
|Donald Golightly|       Medio|Colômbia|         1995|
+----------------+------------+--------+-------------+
only showing top 5 rows

root
 |-- Nome: string (nullable = true)
 |-- Escolaridade: string (nullable = false)
 |-- Pais: string (nullable = false)
 |-- AnoNascimento: integer (nullable = true)



In [35]:
from pyspark.sql.functions import col

df_select = df_nomes.select("AnoNascimento").where((col("AnoNascimento")>2000) & (col("AnoNascimento")<2101))
df_select.show(5)

+-------------+
|AnoNascimento|
+-------------+
|         2010|
|         2008|
|         2007|
|         2009|
|         2001|
+-------------+
only showing top 5 rows



In [38]:
df_nomes.createOrReplaceTempView("pessoas")
spark.sql("select*from pessoas where AnoNascimento > 2000 and AnoNascimento < 2101").show()

+-----------------+------------+---------------+-------------+
|             Nome|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|    Sheila Maurer|       Medio|         Brasil|         2010|
|   Wilfredo Grant|       Medio|          Chile|         2008|
|  Jerry Chynoweth| Fundamental|       Colômbia|         2007|
|      Donald Vogt|       Medio|      Argentina|         2009|
|    Milton Dillon|       Medio|          Chile|         2001|
|       Ned Tester|       Medio|        Bolívia|         2009|
|      Milton Rowe|       Medio|          Chile|         2008|
|       April Ward|       Medio|Guiana Francesa|         2002|
|    Evelyn Shaver|       Medio|         Guiana|         2006|
|   Kenneth Winter|       Medio|          Chile|         2010|
| Maurice Blizzard| Fundamental|         Guiana|         2010|
|  Suzanne Bullard|       Medio|          Chile|         2003|
|  Pamela Banuelos|       Medio|      Argentina|       

In [39]:
df_select = df_nomes.select("AnoNascimento").where((col("AnoNascimento")>1980) & (col("AnoNascimento")<1994))
df_select.count()

1969792

In [65]:
df_nomes.createOrReplaceTempView("pessoas")
spark.sql("select count(*) as Total from pessoas where AnoNascimento between 1979 and 1995").show()

+-------+
|  Total|
+-------+
|2576053|
+-------+



In [66]:
df_nomes.createOrReplaceTempView("geracoes")
df_geracao = spark.sql("""
    select 
        Pais, 
        case 
            when AnoNascimento between 1944 and 1964 then 'Baby Boomers'
            when AnoNascimento between 1965 and 1979 then 'Geração X'
            when AnoNascimento between 1980 and 1994 then 'Millennials (Geração Y)'
            when AnoNascimento between 1995 and 2015 then 'Geração Z'
        end as Geracao,
        count(*) as Quantidade
    from geracoes
    group by Pais, Geracao
    order by Pais, Geracao, Quantidade
""")
df_geracao.show()

+---------+--------------------+----------+
|     Pais|             Geracao|Quantidade|
+---------+--------------------+----------+
|Argentina|        Baby Boomers|    233451|
|Argentina|           Geração X|    174700|
|Argentina|           Geração Z|    186297|
|Argentina|Millennials (Gera...|    175682|
|  Bolívia|        Baby Boomers|    430215|
|  Bolívia|           Geração X|    323385|
|  Bolívia|           Geração Z|    344794|
|  Bolívia|Millennials (Gera...|    323501|
|   Brasil|        Baby Boomers|    546001|
|   Brasil|           Geração X|    410394|
|   Brasil|           Geração Z|    436846|
|   Brasil|Millennials (Gera...|    410348|
|    Chile|        Baby Boomers|    560960|
|    Chile|           Geração X|    421115|
|    Chile|           Geração Z|    449799|
|    Chile|Millennials (Gera...|    420149|
| Colômbia|        Baby Boomers|    485763|
| Colômbia|           Geração X|    362854|
| Colômbia|           Geração Z|    387549|
| Colômbia|Millennials (Gera...|

In [ ]:
spark.stop()